In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [2]:
train_path = "./train/"

train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2, 
    horizontal_flip = True,
    fill_mode='nearest'
)

train_datagenerator = train_generator.flow_from_directory(
    train_path,
    target_size = (227,227),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 1642 images belonging to 10 classes.


In [3]:
test_data = ImageDataGenerator(rescale=1./255)

In [6]:
test_data_generator = test_data.flow_from_directory(
    "./Test/images/",
)

Found 0 images belonging to 0 classes.


In [7]:
base_model = tensorflow.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(227,227,3)
)

C:\Users\sanid\AppData\Local\Temp\ipykernel_4004\4129834316.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tensorflow.keras.applications.MobileNetV2(


In [8]:
X = base_model.output
X = GlobalAveragePooling2D()(X)
X = Dense(1024, activation='relu')(X)
predictions = Dense(10, activation='softmax')(X)


In [9]:
model = Model(inputs = base_model.input, outputs = predictions)

In [10]:
for layer in base_model.layers:
    layer.trainable = False

    

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 227, 227,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 114, 114,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 114, 114,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 114, 114,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 114, 114,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 114, 114,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 114, 114,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 114, 114,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 114, 114,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 114, 114,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 114, 114,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 114, 114,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 115, 115,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 57, 57,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 57, 57,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 57, 57,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 57, 57,    │      2,304 │ block_1_depthwis

 Total params: 3,579,978 (13.66 MB)

 Trainable params: 1,321,994 (5.04 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [13]:
hist = model.fit(
    train_datagenerator, 
    steps_per_epoch= train_datagenerator.samples // train_datagenerator.batch_size, 
    epochs=15, 
)

Epoch 1/15


c:\Users\sanid\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/51 ━━━━━━━━━━━━━━━━━━━━ 11:35 14s/step - accuracy: 0.0938 - loss: 2.4885

c:\Users\sanid\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


51/51 ━━━━━━━━━━━━━━━━━━━━ 58s 880ms/step - accuracy: 0.4771 - loss: 1.8022
Epoch 2/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.7500 - loss: 0.7482 
Epoch 3/15


c:\Users\sanid\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


51/51 ━━━━━━━━━━━━━━━━━━━━ 33s 599ms/step - accuracy: 0.8411 - loss: 0.4936
Epoch 4/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 120us/step - accuracy: 0.9062 - loss: 0.2782 
Epoch 5/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 32s 587ms/step - accuracy: 0.9108 - loss: 0.3068
Epoch 6/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 140us/step - accuracy: 0.9688 - loss: 0.2355 
Epoch 7/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 33s 602ms/step - accuracy: 0.9256 - loss: 0.2580
Epoch 8/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 140us/step - accuracy: 0.8125 - loss: 0.6137 
Epoch 9/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 33s 600ms/step - accuracy: 0.9224 - loss: 0.2245
Epoch 10/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 140us/step - accuracy: 0.9375 - loss: 0.1847 
Epoch 11/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 33s 593ms/step - accuracy: 0.9485 - loss: 0.1931
Epoch 12/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 140us/step - accuracy: 0.9375 - loss: 0.1404 
Epoch 13/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 33s 603ms/step - accuracy: 0.9259 - loss: 0.1956
Epoch 14/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 120us/st

In [15]:
model.save("pokedex_mobilenetv2-2.keras")

In [18]:
model_ = tensorflow.keras.models.load_model('poekdex.h5')

In [4]:
random_model = tensorflow.keras.models.load_model('bottles.h5')

In [6]:
def image_processing(path):
    img = image.load_img(path, target_size=(224,224))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = img_arr/225.0

    return img_arr

def predict(model, path, classlabel):
    img_arr = image_processing(path)
    predictions = model.predict(img_arr)
    predicted_class = np.argmax(predictions, axis=1)[0]
    return classlabel[predicted_class]


In [7]:
class_labels = ["Beer Bottles", "Plastic Bottles", "Soda Bottles", "Water Bottles", "Wine Bottles"]

In [19]:
pr = predict(random_model, "", class_labels)

1/1 [==============================] - 0s 37ms/step


In [20]:
pr

'Water Bottles'

In [5]:
random_model.predict

Signature:
random_model.predict(
    x,
    batch_size=None,
    verbose='auto',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
Docstring:
Generates output predictions for the input samples.

Computation is done in batches. This method is designed for batch
processing of large numbers of inputs. It is not intended for use inside
of loops that iterate over your data and process small numbers of inputs
at a time.

For small numbers of inputs that fit in one batch,
directly use `__call__()` for faster execution, e.g.,
`model(x)`, or `model(x, training=False)` if you have layers such as
`tf.keras.layers.BatchNormalization` that behave differently during
inference. You may pair the individual model call with a `tf.function`
for additional performance inside your inner loop.
If you need access to numpy array values instead of tensors after your
model call, you can use `tensor.numpy()` to get the numpy array value of
an eager tensor.

In [20]:
def image_processing(path):
    img = image.load_img(path, target_size=(227,227))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = img_arr/225.0

    return img_arr

def predict(model, path, classlabel):
    img_arr = image_processing(path)
    predictions = model.predict(img_arr)
    predicted_class = np.argmax(predictions, axis=1)[0]
    return classlabel[predicted_class]

classlabel = ["Aerodactyl","Bulbasaur","Charmander","Dratini","Fearow","Mewtwo","Pikachu","Psyduck","Spearow","Squirtle"]


In [25]:
predicted_class = predict(model_, "pixel-3316924_1280.jpg", classlabel)

1/1 [==============================] - 0s 69ms/step


In [21]:
predicted_class

NameError: name 'predicted_class' is not defined

In [27]:
import os
test_path = "./Test/images/"
test_image_names = os.listdir("./Test/images/")

In [28]:
results = pd.DataFrame(columns=['Name','Class'])
class_label_dict =   {'Aerodactyl': 0, 'Bulbasaur': 1, 'Charmander': 2, 'Dratini': 3, 'Fearow': 4, 'Mewtwo': 5, 'Pikachu': 6, 'Psyduck': 7, 'Spearow': 8, 'Squirtle': 9}

In [29]:
for i in test_image_names:
    predicted_class = predict(model_, test_path + i, classlabel)
    results = results._append({'Name' : i, 'Class':class_label_dict[predicted_class] }, ignore_index=True)

1/1 [==============================] - 0s 71ms/step


In [1]:
results.to_csv('results.csv', index=False)

NameError: name 'results' is not defined